In [1]:
import os
import sys

from django.utils import timezone

import requests
import json
from datetime import datetime

sys.path.append('/home/abezpalov/daven.io/daven/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'daven.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

from dataminer.models import *

In [2]:
bourse = Bourse.objects.take(**{'name': 'liqui', 'full_name': 'Liqui.io',
                                'base_url': 'https://api.liqui.io/api/3'})
bourse

<Bourse: liqui>

In [3]:
n_ = 0
m_ = 0

pairs = Pair.objects.filter(bourse = bourse)
for n, pair in enumerate(pairs):
    infos = PairJSONInfo.objects.filter(pair = pair)
    for m, info in enumerate(infos):
        n_ = n
        m_ = m
n_, m_

(210, 72)

In [43]:
import numpy as np
import pandas as pd
import json

def pair_infos_to_df(pair):

    # Готовим словарь с массивами
    keys = ['info_id', 'bourse_name', 'pair_id', 'pair_name',
             'ticker_high', 'ticker_low', 'ticker_avg', 'ticker_vol', 'ticker_vol_cur', 'ticker_last',
             'ticker_buy', 'ticker_sell', 'ticker_updated']
    for i in range(2000):
        keys.append('order_ask_{}_price'.format(i))
        keys.append('order_ask_{}_vol'.format(i))
    for i in range(2000):
        keys.append('order_bid_{}_price'.format(i))
        keys.append('order_bid_{}_vol'.format(i))
    for i in range(2000):
        keys.append('trade_{}_ask'.format(i))
        keys.append('trade_{}_bid'.format(i))
        keys.append('trade_{}_price'.format(i))
        keys.append('trade_{}_amount'.format(i))
        keys.append('trade_{}_tid'.format(i))
        keys.append('trade_{}_timestamp'.format(i))
    keys.append('created_at')

    data = {}
    for key in keys:
        data[key] = []

    infos = PairJSONInfo.objects.filter(pair = pair)
    for info in infos:
        # Base info
        # print(info)
        # print(pair)
        data['info_id'].append(str(info.id))
        data['bourse_name'].append(str(pair.bourse.name))
        data['pair_id'].append(str(pair.id))
        data['pair_name'].append(str(pair.name))

        # Ticker
        # print(info.ticker)
        # print(type(info.ticker))
        ticker = json.loads(str(info.ticker).replace("'", '"'))
        data['ticker_high'].append(ticker['high'])
        data['ticker_low'].append(ticker['low'])
        data['ticker_avg'].append(ticker['avg'])
        data['ticker_vol'].append(ticker['vol'])
        data['ticker_vol_cur'].append(ticker['vol_cur'])
        data['ticker_last'].append(ticker['last'])
        data['ticker_buy'].append(ticker['buy'])
        data['ticker_sell'].append(ticker['sell'])
        data['ticker_updated'].append(ticker['updated'])

        # Orders
        orders = json.loads(str(info.orders).replace("'", '"'))

        i = 0
        for order in orders['asks']:
            price, vol = order
            data['order_ask_{}_price'.format(i)].append(price)
            data['order_ask_{}_vol'.format(i)].append(vol)
            i += 1
        while i < 2000:
            data['order_ask_{}_price'.format(i)].append(None)
            data['order_ask_{}_vol'.format(i)].append(None)
            i += 1

        i = 0
        for order in orders['asks']:
            price, vol = order
            data['order_bid_{}_price'.format(i)].append(price)
            data['order_bid_{}_vol'.format(i)].append(vol)
            i += 1
        while i < 2000:
            data['order_bid_{}_price'.format(i)].append(None)
            data['order_bid_{}_vol'.format(i)].append(None)
            i += 1

        # Trades
        trades = json.loads(str(info.trades).replace("'", '"'))
        i = 0
        for trade in trades:
            if trade['type'] == 'ask':
                data['trade_{}_ask'.format(i)].append(True)
                data['trade_{}_bid'.format(i)].append(False)
            else:
                data['trade_{}_ask'.format(i)].append(False)
                data['trade_{}_bid'.format(i)].append(True)
            data['trade_{}_price'.format(i)].append(trade['price'])
            data['trade_{}_amount'.format(i)].append(trade['amount'])
            data['trade_{}_tid'.format(i)].append(trade['tid'])
            data['trade_{}_timestamp'.format(i)].append(trade['timestamp'])
            i += 1
        while i < 2000:
            data['trade_{}_ask'.format(i)].append(False)
            data['trade_{}_bid'.format(i)].append(False)
            data['trade_{}_price'.format(i)].append(None)
            data['trade_{}_amount'.format(i)].append(None)
            data['trade_{}_tid'.format(i)].append(None)
            data['trade_{}_timestamp'.format(i)].append(None)
            i += 1

        data['created_at'.format(i)].append(info.created_at)

    return pd.DataFrame(data, columns = keys)

df = pair_infos_to_df(pairs[n_])
df.head()

,info_id,bourse_name,pair_id,pair_name,ticker_high,ticker_low,ticker_avg,ticker_vol,ticker_vol_cur,ticker_last,...,trade_1998_amount,trade_1998_tid,trade_1998_timestamp,trade_1999_ask,trade_1999_bid,trade_1999_price,trade_1999_amount,trade_1999_tid,trade_1999_timestamp,created_at
0,589ba716-350b-4a47-9c07-515f590f28f6,liqui,380,zrx_usdt,1.202999,0.795544,0.999272,119507.124861,122900.632391,1.161400,...,65.543074,56826575,1515032554,False,True,0.916465,308.646814,56826520,1515032550,2018-01-05 13:12:02.249151+00:00
1,9a0672ba-c57c-45d3-99c5-08ef4ac95b47,liqui,380,zrx_usdt,1.202999,0.795544,0.999272,118467.374617,121746.904345,1.143397,...,79.275616,56827488,1515032660,True,False,0.922880,68.231701,56827467,1515032658,2018-01-05 13:27:04.841309+00:00
2,515b748a-d95b-4498-9dfd-0ba65a7189d0,liqui,380,zrx_usdt,1.202999,0.795544,0.999272,117406.020793,120595.468990,1.153688,...,0.016003,56830729,1515033103,True,False,0.914648,6.385247,56830718,1515033101,2018-01-05 13:37:00.446143+00:00
3,02363424-e4c0-4861-8c32-6bb65925ce89,liqui,380,zrx_usdt,1.202999,0.795544,0.999272,117832.282440,120733.558682,1.160647,...,15.915735,56846881,1515035346,False,True,0.903768,99.999055,56846819,1515035341,2018-01-05 13:52:00.159846+00:00
4,4d2216ee-14bd-4383-a449-0d38161f5b22,liqui,380,zrx_usdt,1.202999,0.795544,0.999272,117710.911167,120577.294314,1.169893,...,130.265100,56847292,1515035382,False,True,0.910094,17.536897,56847286,1515035382,2018-01-05 13:57:03.861233+00:00


In [44]:
df.to_csv('~/daven.io/export/{}.csv'.format(str(pairs[n_])))

In [65]:
date = np.max(df.created_at).split(' ')[0]

pairs = Pair.objects.filter(bourse = bourse)
for n, pair in enumerate(pairs):
    df = pair_infos_to_df(pair)
    df.to_csv('~/export/{} {}.csv.tar.gz'.format(date, str(pair)), compression='gzip', index_label='idx')
    print(n, pair)

0 liqui 1st_btc - hidden
1 liqui 1st_eth - hidden
2 liqui 1st_usdt - hidden
3 liqui adx_btc


KeyboardInterrupt: 

In [64]:
df = pd.read_csv('~/export/{} liqui btc_usdt.csv.tar.gz'.format(date), compression='gzip', index_col='idx')
df.head()

,info_id,bourse_name,pair_id,pair_name,ticker_high,ticker_low,ticker_avg,ticker_vol,ticker_vol_cur,ticker_last,...,trade_1998_amount,trade_1998_tid,trade_1998_timestamp,trade_1999_ask,trade_1999_bid,trade_1999_price,trade_1999_amount,trade_1999_tid,trade_1999_timestamp,created_at
idx,,,,,,,,,,,,,,,,,,,,,
0,0edaa871-7c51-4473-a8cc-c62f953fe132,liqui,249,btc_usdt,16563.579872,11829.677423,14196.628648,3.465411e+06,246.438062,16267.252963,...,0.035262,57402908,1515151137,True,False,16343.551678,0.004629,57402884,1515151133,2018-01-05 13:10:21.255454+00:00
1,81482872-73e9-4ca5-b828-8d19e0af1f13,liqui,249,btc_usdt,16563.579872,11829.677423,14196.628648,3.471834e+06,246.459079,16258.604915,...,0.002630,57406236,1515151768,False,True,16407.213443,0.005922,57406232,1515151768,2018-01-05 13:20:23.303901+00:00
2,9ac72e24-cb7a-425d-83ff-5287c63e8a15,liqui,249,btc_usdt,16563.579872,11829.677423,14196.628648,3.452610e+06,244.344005,16255.501523,...,0.002665,57420386,1515154302,True,False,16345.000000,0.000500,57420381,1515154301,2018-01-05 13:40:20.991051+00:00
3,9352674c-ee5e-45d2-bfbe-6760ebcfec91,liqui,249,btc_usdt,16563.579872,11829.677423,14196.628648,3.432915e+06,242.802934,16420.198146,...,0.006461,57422890,1515154642,True,False,16343.584365,0.007041,57422863,1515154639,2018-01-05 13:45:22.171046+00:00
4,a2454e0a-460b-46c0-8f32-01861c7d8237,liqui,249,btc_usdt,16563.579872,11829.677423,14196.628648,3.411874e+06,240.863930,16371.200938,...,0.001789,57430288,1515155760,False,True,16299.000000,0.003555,57430263,1515155759,2018-01-05 14:00:22.525219+00:00


In [62]:
date = np.max(df.created_at).split(' ')[0]
date

'2018-01-20'